# CIS Data to Google Earth (Real Time)

In [1286]:
# TODO: Functionality with meters 
# TODO: Include shorted status in exception report
# TODO: Delete (250V: Range from Comments) 
# TODO: Go through and add/review comments
# TODO: Find more ways to optimize code
# TODO: For multiple line text make sure to change it to ( enter .... enter )
# TODO: Split lines of code to more readable format

In [1287]:
from __future__ import annotations

# --- Standard Library ---
import os
import shutil
import time
import datetime
import socket
import math
import glob
from pathlib import Path
from collections import OrderedDict
from dataclasses import dataclass, field
from dataclasses import replace

# --- Scientific / Stats ---
import scipy.stats as stats

# --- Data Handling ---
import pandas as pd
import numpy as np
from natsort import natsorted

# --- File Format ---
import simplekml
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, Border, Side

# --- Geospatial ---
from geopy.distance import geodesic

# --- Pandas Settings ---
pd.options.mode.chained_assignment = None

In [1288]:
# Measuring execution time
start_time = time.time()

## Configuration


In [1289]:
# CONFIG SETTINGS
SELECTED_CONFIG = 'KM'

# COLORBLIND FRIENDLY COLOR SCHEME
COLORBLIND_MODE = False

### Dataclass

In [1290]:
@dataclass
class Config:
    # --- Basic Settings ---
    CLIENT: str = "Kinder Morgan"
    PLOT_3D: bool = True
    DATA_VISIBILITY: bool = True
    COLOR_SCHEME: int = 0
    REVERSE: bool = False

    # --- GPS Cutoffs ---
    LOWER_CUTOFF: int = 1
    UPPER_CUTOFF: int = 3

    # --- Elevation Scaling ---
    SCALE_FACTOR: float = 117.64705882352942
    SCALE_PCM: float = 100 / 3
    SCALE_PCM_PERCENT: float = 50

    # --- Icon Scale ---
    ICON_SCALE: float = 0.2

    # --- Potential Thresholds ---
    POTENTIAL_850 = -0.85
    POTENTIAL_1200 = -1.2

    # --- Icon URLs (to be set in __post_init__) ---
    ICON_ON: str = field(init=False)
    ICON_OFF: str = field(init=False)
    ICON_NATIVE: str = field(init=False)
    ICON_1200: str = field(init=False)
    ICON_850: str = field(init=False)
    ICON_COMMENTS: str = field(init=False)
    ICON_ACVG: str = field(init=False)
    ICON_PCM: str = field(init=False)
    ICON_PCM_PERCENT: str = field(init=False)

    def __post_init__(self):
        # Default (0)
        icons = {
            0: {
                'ICON_ON': 'https://img.icons8.com/ios-filled/50/FC9CFF/filled-circle.png',
                'ICON_OFF': 'https://img.icons8.com/ios-filled/50/00FF00/filled-circle.png',
                'ICON_NATIVE': 'https://img.icons8.com/ios-filled/50/175082/filled-circle.png',
                'ICON_1200': 'https://img.icons8.com/ios-filled/50/7950F2/filled-circle.png',
                'ICON_850': 'https://img.icons8.com/ios-filled/50/F25081/filled-circle.png',
            },
            1: {
                'ICON_ON': 'https://img.icons8.com/ios-filled/50/3D8D7A/filled-circle.png',
                'ICON_OFF': 'https://img.icons8.com/ios-filled/50/FFFACD/filled-circle.png',
                'ICON_NATIVE': 'https://img.icons8.com/ios-filled/50/FFF6DA/filled-circle.png',
                'ICON_1200': 'https://img.icons8.com/ios-filled/50/2D336B/filled-circle.png',
                'ICON_850': 'https://img.icons8.com/ios-filled/50/A94A4A/filled-circle.png',
            },
            2: {
                'ICON_ON': 'https://img.icons8.com/ios-filled/50/87FF00/filled-circle.png',
                'ICON_OFF': 'https://img.icons8.com/ios-filled/50/0079F7/filled-circle.png',
                'ICON_NATIVE': 'https://img.icons8.com/ios-filled/50/FFF6DA/filled-circle.png',
                'ICON_1200': 'https://img.icons8.com/ios-filled/50/00FFD4/filled-circle.png',
                'ICON_850': 'https://img.icons8.com/ios-filled/50/FFFFFF/filled-circle.png',
            }
        }

        selected = icons.get(self.COLOR_SCHEME, icons[0])  # fallback to 0
        self.ICON_ON = selected['ICON_ON']
        self.ICON_OFF = selected['ICON_OFF']
        self.ICON_NATIVE = selected['ICON_NATIVE']
        self.ICON_1200 = selected['ICON_1200']
        self.ICON_850 = selected['ICON_850']

        # Shared across all schemes
        self.ICON_COMMENTS = 'https://img.icons8.com/ultraviolet/80/000000/comments.png'
        self.ICON_ACVG = 'https://img.icons8.com/material-rounded/50/6BFF00/sine.png'
        self.ICON_PCM = 'https://img.icons8.com/ios-filled/50/FFFFFF/lightning-bolt--v1.png'
        self.ICON_PCM_PERCENT = (
            'https://img.icons8.com/fluency-systems-regular/50/FFFFFF/percentage-circle.png'
        )

### Presets

In [1291]:
KM = Config(CLIENT='Kinder Morgan')
KM_1 = Config(CLIENT='Kinder Morgan', COLOR_SCHEME=1)
ENB = Config(CLIENT='Enbridge')

In [1292]:
# Config map
config_map = {
    'KM': KM,
    'KM_1': KM_1,
    'ENB': ENB,
}

# Select config (Default = KM)
config = config_map.get(SELECTED_CONFIG, KM)

# Colorblind mode
if COLORBLIND_MODE:
    config = replace(config, COLOR_SCHEME=2)

## Function Definitions

### Exception Report Dataframe

In [1306]:
def exception_report_df(
        export_name: str,
        *,
        potential: str,
) -> tuple[pd.DataFrame, str]:
    """
    Load and process a '.csv' file for an exception report.

    Parameters
    ----------
    export_name : str
        Base name of the CSV file (without '(Modified).csv').
    potential : str
        Either 'On Potential' or 'Off Potential' (also accepts 'on'/'off', case-insensitive).

    Returns
    -------
    tuple of (pd.DataFrame, str)
        Processed DataFrame with numeric coercion, filtering, and stable indexing.  
        Column name of potential ('On Potential' or 'Off Potential').

    Raises
    ------
    ValueError
        If `potential` is not 'on'/'off' or a valid full column name.
    """
    # Map/validate potential inside the function
    pot_map = {'on': 'On Potential', 'off': 'Off Potential'}
    pot_key = potential.strip().lower()
    potential = pot_map.get(pot_key, potential)

    # Error handling
    if potential not in {'On Potential', 'Off Potential'}:
        raise ValueError(
            f"'potential' must be 'On Potential', 'Off Potential' "
            f"or shorthand 'on'/'off'; got {potential!r}"
        )

    file_dir = Path(DATA_DIR)  # TODO: Update directories to Path
    file_path = file_dir / f'{export_name}(Modified).csv'
    relevant_cols = [
        'Station',
        'Stationing (ft)',
        'Latitude',
        'Longitude',
        potential,
        'ACVG Indication (dBV)'
    ]

    # Read only relevant columns
    df_out = pd.read_csv(file_path, usecols=lambda c: c in relevant_cols)

    # Drop rows with missing or zero potential
    df_out = df_out.loc[df_out[potential].notna() & (df_out[potential] != 0)].copy()

    # Column order
    df_out = df_out.reindex(columns=[c for c in relevant_cols if c in df_out.columns])

    return df_out, potential

### Exception Report Crossings

In [1294]:
def exception_report_crossings(
        df_in: pd.DataFrame,
        *,
        potential: str,
        threshold: float
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Identify and filter threshold crossing events in potential data, removing single-point spikes
    and returning both cleaned data and detected outliers.

    Notes
    -----
    1. Flags rows where the `potential` column meets or exceeds the given `threshold`.
    2. Calculates transitions between rows:
        - +1 = rising edge (below threshold → above threshold)
        - -1 = falling edge (above threshold → below threshold)
        - 0  = no change.
    3. Returns:
        - df_out : Cleaned dataframe of threshold crossings without single-point spikes.
        - df_outliers : Dataframe of detected outlier spike events.

    Parameters
    ----------
    df_in : pd.DataFrame
        Input dataframe containing time series data and a `Potential` column.
    potential : str
        Name of the column containing potential values to evaluate.
    threshold : float
        Threshold value for determining crossings.

    Returns
    -------
    (pd.DataFrame, pd.DataFrame)
        df_out, df_outliers
    """
    # 'On Potential' column
    potential_col = df_in[potential]

    # Convert to float
    threshold = float(threshold)

    # Boolean mask indicating which rows have potential values at or above the threshold
    point = potential_col >= threshold

    # Boolean mask of the previous row's threshold state (False for the first row)
    point_prev = point.shift(1, fill_value=False)

    # Convert boolean masks to integers (True → 1, False → 0) for arithmetic comparison
    point_int = point.astype(int)
    point_prev_int = point_prev.astype(int)

    # +1 = rising ('-.), -1 = falling (.-'), 0 = no change
    transition = point_int - point_prev_int

    # Replace rows where transition == -1 (falling) with the previous row
    df_out = df_in.copy()
    mask_fall = (transition == -1)
    df_out.loc[mask_fall, :] = df_out.shift(1).loc[mask_fall, :]

    # Detect a (1, -1) transition pair, which represents a single-point spike:
    #   - start_of_pair: marks the first row of the spike (rising followed falling)
    #   - to_drop: marks both the spike's start row (rising) and the immediate next row (falling)
    #   - fillna(False): ensures no NaN values in the mask (important at dataframe edges)
    start_of_pair = (transition == 1) & (transition.shift(-1) == -1)
    to_drop = start_of_pair | start_of_pair.shift(1)
    to_drop = to_drop.fillna(False)

    # Keep rows
    to_keep = (transition != 0) & (~to_drop)

    # Outlier dataframe (drop duplicate stations)
    df_outliers = df_out.loc[to_drop].drop_duplicates(subset="Station").reset_index(drop=True)

    # Crossings dataframe
    df_out = df_out.loc[to_keep].reset_index(drop=True)
    
    
    # If df_out has an odd number of rows (Deal with last value), append the last row from df_in
    if len(df_out) % 2 != 0:
        df_out = pd.concat([df_out, df_in.tail(1)], ignore_index=True)

    return df_out, df_outliers

### Exception Report To Excel

In [1295]:
def exception_report_to_excel(
        df: pd.DataFrame,
        *,
        acvg_threshold: float = 45.0,
        interval_closed: str = 'both',
        acvg_col: str = 'ACVG Indication (dBV)'
) -> pd.DataFrame:
    # Pair rows into segments
    starts = df.iloc[::2].reset_index(drop=True)
    ends = df.iloc[1::2].reset_index(drop=True)
    nseg = len(starts)

    # Segment lengths
    length = ends['Station'] - starts['Station']

    # Create empty series
    acvg_max = pd.Series(pd.NA, index=range(nseg), dtype='Float64')

    # ACVG max >= threshold in [start, end]
    if {'Station', acvg_col} <= set(df.columns) and nseg:
        # ACVG slice of dataframe above threshold
        df_acvg = df[['Station', acvg_col]].copy()
        df_acvg = df_acvg.loc[df_acvg[acvg_col] >= acvg_threshold]

        if not df_acvg.empty:
            # Build an IntervalIndex representing each segment's station range
            intervals = pd.IntervalIndex.from_arrays(
                starts['Station'],
                ends['Station'],
                closed=interval_closed,
            )

            # For each ACVG measurement, find the index of the segment it falls into
            seg_idx = intervals.get_indexer(df_acvg['Station'])

            # Add the segment index (_seg) to the ACVG DataFrame
            df_acvg = df_acvg.assign(_seg=seg_idx).loc[lambda x: x['_seg'] >= 0]

            # Group the ACVG data by segment index and get the maximum ACVG value per segment
            acvg_max_map = df_acvg.groupby('_seg')[acvg_col].max()

            # Fills max acvg value for the segment
            acvg_max = (
                pd.Series(range(nseg))
                .map(acvg_max_map)
                .fillna('')  # Replace NaN with empty string
                .astype('object')  # Store strings and numbers together
            )

    # Build final excel ready frame
    data = {
        # Start
        'Station Number': starts['Stationing (ft)'],
        'Latitude': starts['Latitude'],
        'Longitude': starts['Longitude'],

        # End
        'Station Number_end': ends['Stationing (ft)'],
        'Latitude_end': ends['Latitude'],
        'Longitude_end': ends['Longitude'],

        # Derived
        'Length (ft)': length,
        'Comments': pd.Series(pd.NA, index=range(nseg), dtype='string'),
    }

    # Conditionally add ACVG
    if {'Station', acvg_col} <= set(df.columns):
        data['ACVG Max (dB/V)'] = acvg_max

    # Build DataFrame
    df_out = pd.DataFrame(data)

    # Define final column order dynamically
    cols = [
        'Station Number', 'Latitude', 'Longitude',
        'Station Number_end', 'Latitude_end', 'Longitude_end',
        'Length (ft)',
    ]

    if 'ACVG Max (dB/V)' in df_out.columns:
        cols.append('ACVG Max (dB/V)')

    cols.append('Comments')

    # Apply column order
    df_out = df_out[cols]

    # Rename _end columns
    df_out = df_out.rename(columns={
        'Station Number_end': 'Station Number',
        'Latitude_end': 'Latitude',
        'Longitude_end': 'Longitude',
    })

    return df_out

### Exception Report Format Excel

In [1296]:
def exception_report_format_excel(
        file_path: Path,
        export_name: str,
        original_name: str,
        client: str,
        total_miles_list: list[float],
        last_index: int,
        df_lengths: pd.DataFrame,
        sheet_name: str = "Less Negative than -0.85V (On)",
) -> None:
    """
    Apply header text, formatting, merges, alignment, borders, and column widths
    to the Exception Report Excel sheet.
    """
    wb = load_workbook(file_path)
    sheet = wb[sheet_name]

    # --- Header Text ---
    sheet.cell(1, 1).value = f"{client} ({original_name.split(' (SN')[0]})"
    sheet.cell(2, 1).value = (
        f"{original_name.split('SN')[1].split()[0]} to "
        f"{original_name.split('SN')[2].split(').')[0]}"
    )
    sheet.cell(4, 1).value = "Start"
    sheet.cell(4, 4).value = "End"

    total_feet = round(sum(total_miles_list) * 5280)
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )

    total_length = round(df_lengths["Length (ft)"].sum(), 2)
    length_percent = round(total_length / (sum(total_miles_list) * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance less negative than -0.85V 'On' = "
        f"{total_length} feet ({length_percent} %)"
    )

    # --- Fonts ---
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    for row, col in [(4, 1), (4, 4), (4, 7), (4, 8), (4, 9), (last_index + 7, 1),
                     (last_index + 8, 1)]:
        sheet.cell(row, col).font = Font(bold=True)

    # --- Merges ---
    sheet.merge_cells("A1:I1")
    sheet.merge_cells("A2:I2")
    sheet.merge_cells("A4:C4")
    sheet.merge_cells("D4:F4")
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells("G4:G5")
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells("H4:H5")
    sheet.cell(4, 9).value = sheet.cell(5, 9).value
    sheet.merge_cells("I4:I5")

    # --- Alignment ---
    for row in sheet[f"A1:I{last_index + 5}"]:
        for cell in row:
            cell.alignment = Alignment(horizontal="center", vertical="center")

    # --- Borders ---
    thin = Side(border_style="thin", color="000000")
    white_border = Side(border_style="thin", color="FFFFFF")
    for i, row in enumerate(sheet[f"A1:I{last_index + 5}"]):
        # Special 3rd row
        if i == 2:
            for cell in row:
                cell.border = Border(left=white_border, right=white_border)
            continue
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

    # --- Column Widths ---
    for col_letter in "ABCDEFGH":
        sheet.column_dimensions[col_letter].width = 15
    sheet.column_dimensions["I"].width = 50

    wb.save(file_path)

## Directories & Paths

In [1297]:
# TODO: Dataclass for directories & paths
# TODO: Change from os.path.join(...) to Path(...)

In [1298]:
# Directories
ROOT_DIR = os.path.abspath('../')
DESKTOP_DIR = os.path.join(os.path.expanduser("~"), 'Desktop')
ORIGINAL_DATA_DIR = os.path.join(ROOT_DIR, 'Original Data')
OUTPUT_DIR = os.path.join(DESKTOP_DIR, 'Output')
DATA_DIR = os.path.join(OUTPUT_DIR, 'Data')
KMZ_DIR = os.path.join(OUTPUT_DIR, 'KMZ')
LOGS_DIR = os.path.join(OUTPUT_DIR, 'Logs')

In [1299]:
# Paths
OUTPUT_DUPLICATES_PATH = os.path.join(OUTPUT_DIR, 'Duplicate GPS Pairs.csv')

In [1300]:
# Replaces output directory with an empty one
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)
os.makedirs(DATA_DIR)
os.makedirs(KMZ_DIR)
os.makedirs(LOGS_DIR)

# Data

## Original Data

In [1301]:
# List for files excluding ones starting with '.'
original_data_list = [file for file in os.listdir(ORIGINAL_DATA_DIR) if not file.startswith('.')]

# Modify name of original data files
for idx, file_name in enumerate(original_data_list):
    # Original file name
    original_name = os.path.join(ORIGINAL_DATA_DIR, file_name)

    # Modify file name
    if '(SN' not in file_name and ').csv' not in file_name:
        file_name = file_name.replace(' SN', ' (SN')
        file_name = file_name.replace('.csv', ').csv')
        file_name = file_name.replace('.CSV', ').csv')
        file_name = file_name.replace('.DAT', ').csv')
        file_name = file_name.replace('TO (SN', 'TO SN')

        # New file name
        new_name = os.path.join(ORIGINAL_DATA_DIR, file_name)

        # Rename files in directory
        os.rename(original_name, new_name)

        # Raname files in list
        original_data_list[idx] = new_name.split('Original Data/')[1]

# Sort
original_data_list = natsorted(original_data_list)

# Export name list
export_name_list = natsorted([name.split('.csv')[0] for name in original_data_list])

## Dataframe

In [1302]:
log_dict = {}
dfs = {}
total_miles_list = []  # List for total miles per '.csv' file
total_rows_dropped = []  # List for total rows dropped that don't have GPS coordinates
starting_stationing_list = []  # List for starting stationing per '.csv' file
i = 0  # Loop counter for station numbers
j = 0  # List counter for '.csv' files

# Goes through each '.csv' in the folder
while j < len(original_data_list):
    # ----------------------------------------------------------------------------------------------
    # LOG DICTIONARY
    # ----------------------------------------------------------------------------------------------

    log_dict[export_name_list[j]] = {
        'Dataframe': {
            'Rows Dropped': None
        },
        'Statistics': {
            'Mean': None,
            'Mode': None,
            'Std': None,
            'Total Count': None,
            'Cutoff Count': None,
            'Outliers Count': None,
            'Duplicates Count': None
        },
        'Measurements': {
            'On': None,
            'Off': None
        }
    }

    # ----------------------------------------------------------------------------------------------
    # DATAFRAME
    # ----------------------------------------------------------------------------------------------

    # Create dataframe
    df_cis = pd.read_csv(os.path.join(ORIGINAL_DATA_DIR, original_data_list[j]))
    df_cis.to_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Original).csv"), index=False)

    # Rename columns
    rename_columns_dict = {
        'records': 'Station',
        'milepost': 'Station',
        'ps on': 'On Potential',
        'structureps': 'On Potential',
        'ps off': 'Off Potential',
        'structureirf': 'Off Potential',
        'comment': 'Comments',
        'comments': 'Comments',
        'locationdescription': 'Comments',
        'latitude': 'Latitude',
        'longitude': 'Longitude',
        'ACVG Indication (dB_V)': 'ACVG Indication (dBV)'
    }

    # Replace original names to correct format
    df_cis.columns = [rename_columns_dict.get(x.lower(), x) for x in df_cis.columns]

    # Columns for dataframe
    cols = ['Station', 'On Potential', 'Off Potential', 'Native',
            'Comments', 'Latitude', 'Longitude',
            'ACVG Indication (dBV)', 'ACVG Notes', 'PCM Data (Amps)', 'PCM % Change']

    # Create dataframe from columns that exist
    df_cis = df_cis[df_cis.columns.intersection(cols)]

    # Reversing rows for flipped SN
    if config.REVERSE:
        # Reversing rows
        df_cis.iloc[:, :] = df_cis.iloc[:, :].values[::-1]

        # Reordering
        df_cis['Station'] = abs(max(df_cis['Station']) - df_cis['Station'])

    # Desired columns
    cols = ['On Potential', 'Off Potential', 'Native']

    for col in cols:
        if col in df_cis.columns:
            # Delete rows that CIS was skipped and reset the index
            df_cis = df_cis.loc[df_cis[col] != 'SKIP'].reset_index(drop=True)

            # Convert object columns to numbers
            df_cis[col] = pd.to_numeric(df_cis[col], errors='coerce')

            # Replace empty values with 0
            df_cis[col] = df_cis[col].replace(np.nan, 0)

            # Convert positive values to negative
            df_cis[col] = df_cis[col].abs() * (-1)

            # Replace exact values
            df_cis[col] = df_cis[col].replace(-0.85, -0.850001)
            df_cis[col] = df_cis[col].replace(-1.2, -1.20001)

        # Divide by 1000 if needed
        if col in df_cis.columns and abs(df_cis[col].mean()) > 100:
            df_cis[col] /= 1000

    # Trim white space from comments
    df_cis['Comments'] = df_cis['Comments'].str.strip()

    # Create 'Distance (ft)' column
    df_cis['Distance (ft)'] = 0.00

    # Initial rows 
    last_index2 = df_cis.last_valid_index()

    # Drop rows that don't have GPS coordinates
    df_cis['Latitude'] = df_cis['Latitude'].replace('', np.nan)
    df_cis.dropna(subset=['Latitude'], inplace=True)
    df_cis.reset_index(drop=True, inplace=True)
    last_index = df_cis.last_valid_index()
    total_rows_dropped.append(last_index2 - last_index)

    log_dict[export_name_list[j]]['Dataframe']['Rows Dropped'] = total_rows_dropped[j]

    # Records total miles in a list
    total_miles_list.append(max(df_cis['Station']) / 5280)

    # ----------------------------------------------------------------------------------------------
    # ACVG, PCM & PCM %
    # ----------------------------------------------------------------------------------------------

    if 'ACVG Indication (dBV)' in df_cis.columns:
        # Convert object columns to numbers
        df_cis['ACVG Indication (dBV)'] = pd.to_numeric(
            df_cis['ACVG Indication (dBV)'], errors='coerce'
        )

        # Replace empty values with 0
        df_cis['ACVG Indication (dBV)'] = df_cis['ACVG Indication (dBV)'].replace(np.nan, 0)

        # Trim white space from comments
        df_cis['ACVG Notes'] = df_cis['ACVG Notes'].str.strip()

    if 'PCM Data (Amps)' in df_cis.columns:
        # Convert object columns to numbers
        df_cis['PCM Data (Amps)'] = pd.to_numeric(df_cis['PCM Data (Amps)'], errors='coerce')

        # Replace empty values with 0
        df_cis['PCM Data (Amps)'] = df_cis['PCM Data (Amps)'].replace(np.nan, 0)

    if 'PCM % Change' in df_cis.columns:
        # Convert object columns to numbers
        df_cis['PCM % Change'] = pd.to_numeric(df_cis['PCM % Change'], errors='coerce')

        # Replace empty values with 0
        df_cis['PCM % Change'] = df_cis['PCM % Change'].replace(np.nan, 0)

        # Absolute value
        df_cis['PCM % Change'] = abs(df_cis['PCM % Change'])

    # ----------------------------------------------------------------------------------------------
    # STATION NUMBERS
    # ----------------------------------------------------------------------------------------------

    # Extract starting station number from '.csv' title
    starting_stationing_list.append(float(original_data_list[j].split('SN')[1].split()[0].
                                          replace('+', '')))

    # Add 'Stationing (ft)' column to hold calculated stationing values
    df_cis['Stationing (ft)'] = ''

    # Infers actual station number (XX+XX)
    while i <= last_index:
        current_station = df_cis.at[i, 'Station']
        stationing_ft = str(starting_stationing_list[j] + current_station).split('.')[0]
        string_length = len(stationing_ft)

        if -100 < (starting_stationing_list[j] + current_station) < 0:
            station_value = stationing_ft.replace('-', '')
            df_cis.at[i, 'Stationing (ft)'] = (
                f" -0{station_value[:string_length - 2]}+{station_value[string_length - 2:]}")

        elif 0 <= (starting_stationing_list[j] + current_station) < 100:
            df_cis.at[i, 'Stationing (ft)'] = (
                f" 0{stationing_ft[:string_length - 2]}+{stationing_ft[string_length - 2:]}")

        else:
            df_cis.at[i, 'Stationing (ft)'] = (
                f"{stationing_ft[:string_length - 2]}+{stationing_ft[string_length - 2:]}")

        # Last row exception
        if i != last_index:
            # Calculates distance between GPS coordinates and stationing (ft)
            try:
                df_cis.at[i + 1, 'Distance (ft)'] = geodesic(
                    [df_cis.at[i, 'Latitude'], df_cis.at[i, 'Longitude']],
                    [df_cis.at[i + 1, 'Latitude'], df_cis.at[i + 1, 'Longitude']]).ft

            except ZeroDivisionError:
                df_cis.at[i + 1, 'Distance (ft)'] = 0

        # Counters
        i += 1

    # ----------------------------------------------------------------------------------------------
    # STATISTICS # TODO: Go through statistics to see what is valuable
    # ----------------------------------------------------------------------------------------------

    df_cis_stats = df_cis.copy()
    df_cis_stats = df_cis_stats[(config.LOWER_CUTOFF < df_cis_stats['Station'].diff().abs()) &
                                (df_cis_stats['Station'].diff().abs() <= config.UPPER_CUTOFF)]

    # Mean
    log_dict[export_name_list[j]]['Statistics']['Mean'] = (
        df_cis_stats['Distance (ft)'][1:].mean())

    # Mode
    log_dict[export_name_list[j]]['Statistics']['Mode'] = (
        df_cis_stats['Distance (ft)'].round(1).mode()[0])

    # Standard deviation
    log_dict[export_name_list[j]]['Statistics']['Std'] = (
        df_cis_stats['Distance (ft)'][1:].std())

    # Z-score (Identify outliers)
    z_scores = np.abs(stats.zscore(df_cis_stats['Distance (ft)']))
    df_cis_stats['z_scores'] = z_scores
    df_outliers = df_cis_stats[df_cis_stats['z_scores'] > 2.576][['Stationing (ft)',
                                                                  'Distance (ft)']]

    # Total count of data within the distance cutoff
    log_dict[export_name_list[j]]['Statistics']['Total Count'] = (
        df_cis_stats['Distance (ft)'].round(1).count())

    # Count of data that fall outside the cutoff
    log_dict[export_name_list[j]]['Statistics']['Cutoff Count'] = (
        df_cis_stats[(config.LOWER_CUTOFF < df_cis_stats['Distance (ft)']) &
                     (df_cis_stats['Distance (ft)'] <=
                      config.UPPER_CUTOFF)]['Distance (ft)'].count())

    # Count of data that are outliers
    log_dict[export_name_list[j]]['Statistics']['Outliers Count'] = (
        df_outliers['Distance (ft)'].count())

    # ----------------------------------------------------------------------------------------------
    # EXPORTS
    # ----------------------------------------------------------------------------------------------

    # Export modified data to csv
    df_cis.to_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"), index=False)

    # Export outlier data to csv
    df_outliers.to_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(GPS Outliers).csv"),
                       index=False)

    # Counters
    i = 0
    j += 1

In [1303]:
df = exception_report_df('Crawford Line 3 (SN1066+59 TO SN1882+91)', potential='On Potential')[0]
potential_col = exception_report_df('Crawford Line 3 (SN1066+59 TO SN1882+91)',
                                    potential='On Potential')[1]
# df_2 = exception_report_mark_crossings(df, potential=potential_col, threshold=Config.POTENTIAL_850)
df_2 = exception_report_crossings(df, potential=potential_col, threshold=Config.POTENTIAL_850)[0].head(25)
df_3 = exception_report_crossings(df, potential=potential_col, threshold=Config.POTENTIAL_850)[1].head(25)
df_2

,Station,Stationing (ft),Latitude,Longitude,On Potential
0,0.0,1066+59,41.701096,-87.962357,-0.25
1,2.5,1066+61,41.701096,-87.962353,-0.25
2,10.0,1066+69,41.701103,-87.962326,-0.25
3,12.5,1066+71,41.701103,-87.962324,-0.25
4,40.0,1066+99,41.701107,-87.962246,-0.25
5,50.0,1067+09,41.701110,-87.962213,-0.55
6,65.0,1067+24,41.701111,-87.962173,-0.25
7,70.0,1067+29,41.701111,-87.962151,-0.25
8,64867.5,1715+26,41.822258,-87.738373,-0.10
9,64877.5,1715+36,41.822268,-87.738326,-0.25


In [1304]:
df_3

,Station,Stationing (ft),Latitude,Longitude,On Potential
0,60.0,1067+19,41.701111,-87.962189,-0.25
1,95.0,1067+54,41.701115,-87.962051,-0.25


In [1305]:
exception_report_to_excel(df_2)

,Station Number,Latitude,Longitude,Station Number,Latitude,Longitude,Length (ft),Comments
0,1066+59,41.701096,-87.962357,1066+61,41.701096,-87.962353,2.5,<NA>
1,1066+69,41.701103,-87.962326,1066+71,41.701103,-87.962324,2.5,<NA>
2,1066+99,41.701107,-87.962246,1067+09,41.701110,-87.962213,10.0,<NA>
3,1067+24,41.701111,-87.962173,1067+29,41.701111,-87.962151,5.0,<NA>
4,1715+26,41.822258,-87.738373,1715+36,41.822268,-87.738326,10.0,<NA>


# RUN CODE TO HERE

# Google Earth

## Folder Structure

In [ ]:
kmz_path_list = []  # List for kmz paths
cis_kmz = []  # List for '.kmz' files
type_folders = []  # List for folders
total_folders = 9
current_mile = 1
k = 0  # List counter for folders
j = 0  # List counter for '.csv' files
i = 0  # List counter for '.kmz' files based on miles

# Create '.kmz' files per mile #
while j < len(total_miles_list):
    miles_remaining = round(total_miles_list[j], 2)

    # Create folders
    while miles_remaining > 0:
        # Separate folders by mile
        cis_kmz.append(f"{export_name_list[j]} (Mile {current_mile})")
        cis_kmz[i] = simplekml.Kml()

        # Create list placeholders for folders
        for l in range(total_folders):
            type_folders.append('')

        type_folders[k + 0] = cis_kmz[i].newfolder(name='On')
        type_folders[k + 1] = cis_kmz[i].newfolder(name='Off')

        if 'Native' in df_cis.columns:
            type_folders[k + 2] = cis_kmz[i].newfolder(name='Native')

        type_folders[k + 3] = cis_kmz[i].newfolder(name='Comments')
        type_folders[k + 4] = cis_kmz[i].newfolder(name='-1.2 V')
        type_folders[k + 5] = cis_kmz[i].newfolder(name='-0.85 V')

        if 'ACVG Indication (dBV)' in df_cis.columns:
            type_folders[k + 6] = cis_kmz[i].newfolder(name='ACVG Indication')

        if 'PCM Data (Amps)' in df_cis.columns:
            type_folders[k + 7] = cis_kmz[i].newfolder(name='PCM (Amps)')

        if 'PCM % Change' in df_cis.columns:
            type_folders[k + 8] = cis_kmz[i].newfolder(name='PCM (%)')

        # Directory name
        kmz_path_list.append(os.path.join(KMZ_DIR, export_name_list[j]))

        # Create directories for each file  
        if not os.path.exists(kmz_path_list[i]):
            os.makedirs(kmz_path_list[i])

        # Export to '.kmz'
        kmz_name = f"{export_name_list[j]} (Mile {current_mile}).kmz"
        cis_kmz[i].savekmz(os.path.join(kmz_path_list[i], kmz_name))

        # Transform directories to paths for easier access
        kmz_path_list[i] = os.path.join(kmz_path_list[i], kmz_name)

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        i += 1
        k += total_folders

    # Counters
    j += 1
    current_mile = 1

## On

In [ ]:
feet_counter = 5280
style = simplekml.Style()
i = 0  # Loop counter for rows
j = 0  # Loop counter for '.csv' files
k = 0  # Loop counter for type folders (0 = 'On')
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and 'On Potential' in df_cis.columns and config.PLOT_3D:
    df_cis_on = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    df_cis_on = df_cis_on[df_cis_on['On Potential'] != 0]
    df_cis_on = df_cis_on[['Station', 'Stationing (ft)', 'Comments', 'Latitude',
                           'Longitude', 'On Potential']].reset_index(drop=True)
    df_cis_on['On Potential'] = df_cis_on['On Potential'] * (-1)
    last_index = df_cis_on.last_valid_index()
    log_dict[export_name_list[j]]['Measurements']['On'] = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_on.at[i, 'Station'] < feet_counter:
            # Break loop if there are no 'On' potentials
            if df_cis_on.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_on.at[i, 'On Potential'] * (-1),
                                           visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential (On): -{df_cis_on.at[i, 'On Potential']} V\n"
                f"Longitude: {df_cis_on.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_on.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_on.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_on.at[i, 'Longitude'], df_cis_on.at[i, 'Latitude'],
                           df_cis_on.at[i, 'On Potential'] * config.SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_ON
            pnt.style.iconstyle.scale = config.ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Export duplicate GPS coordinates to csv
    if config.PLOT_3D:
        df_duplicates = df_cis_on[df_cis_on.duplicated(subset=['Latitude', 'Longitude'],
                                                       keep=False)]
        df_duplicates.to_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Duplicate GPS).csv"),
                             index=False)
        log_dict[export_name_list[j]]['Statistics']['Duplicates Count'] = (
            df_duplicates['Station'].value_counts().sum())

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## Off

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 1  # Loop counter for type folders (1 = 'Off')
kmz_file = 0
off_measurements = 0

# Goes through all '.csv' files
while j < len(original_data_list) and 'Off Potential' in df_cis.columns and config.PLOT_3D:
    df_cis_off = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    df_cis_off = df_cis_off[df_cis_off['Off Potential'] != 0]
    df_cis_off = df_cis_off[['Station', 'Stationing (ft)', 'Latitude', 'Longitude',
                             'Off Potential']].reset_index(drop=True)
    df_cis_off['Off Potential'] = df_cis_off['Off Potential'] * (-1)
    last_index = df_cis_off.last_valid_index()
    log_dict[export_name_list[j]]['Measurements']['Off'] = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_off.at[i, 'Station'] < feet_counter:
            # Break loop if there are no 'Off' potentials
            if df_cis_off.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_off.at[i, 'Off Potential'] * (-1),
                                           visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential (Off): -{df_cis_off.at[i, 'Off Potential']} V\n"
                f"Longitude: {df_cis_off.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_off.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_off.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_off.at[i, 'Longitude'], df_cis_off.at[i, 'Latitude'],
                           df_cis_off.at[i, 'Off Potential'] * config.SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_OFF
            pnt.style.iconstyle.scale = config.ICON_SCALE
            pnt.style.linestyle.width = 0.01
            pnt.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, round(255 * 0.15))
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 1

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## Native

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 2  # Loop counter for type folders (2 = 'Native')
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and 'Native' in df_cis.columns and config.PLOT_3D:
    df_cis_native = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    df_cis_native = df_cis_native[df_cis_native['Native'] != 0]
    df_cis_native = df_cis_native[['Station', 'Stationing (ft)', 'Latitude', 'Longitude',
                                   'Native']].reset_index(drop=True)
    df_cis_native['Native'] = df_cis_native['Native'] * (-1)
    last_index = df_cis_native.last_valid_index()
    off_measurements = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_native.at[i, 'Station'] < feet_counter:
            # Break loop if there are no 'Native' potentials
            if df_cis_native.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_native.at[i, 'Native'] * (-1),
                                           visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential (Native): -{df_cis_native.at[i, 'Native']} V\n"
                f"Longitude: {df_cis_native.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_native.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_native.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_native.at[i, 'Longitude'], df_cis_native.at[i, 'Latitude'],
                           df_cis_native.at[i, 'Native'] * config.SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_NATIVE
            pnt.style.iconstyle.scale = config.ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## Comments

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 3  # Loop counter for type folders (3 = 'Comments')
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and 'Comments' in df_cis.columns and config.PLOT_3D:
    df_cis_comments = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    df_cis_comments.dropna(subset=['Comments'], inplace=True)
    df_cis_comments.reset_index(drop=True, inplace=True)
    last_index = df_cis_comments.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_comments.at[i, 'Station'] < feet_counter:
            # Break loop if there are no comments
            if df_cis_comments.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_comments.at[i, 'Comments'],
                                           visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = ''

            if 'On Potential' in df_cis.columns:
                potential_on = df_cis_comments.at[i, 'On Potential'].__str__() + ' V'
                pnt.style.balloonstyle.text += f"Potential (On): {potential_on}\n"

            if 'Off Potential' in df_cis.columns:
                potential_off = df_cis_comments.at[i, 'Off Potential'].__str__() + ' V'
                pnt.style.balloonstyle.text += f"Potential (Off): {potential_off}\n"

            if 'Native' in df_cis.columns:
                potential_native = df_cis_comments.at[i, 'Native'].__str__() + ' V'
                pnt.style.balloonstyle.text += f"Potential (Native): {potential_native}\n"

            pnt.style.balloonstyle.text += (
                f"Longitude: {df_cis_comments.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_comments.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_comments.at[i, 'Stationing (ft)']}\n"
                f"Comment: {df_cis_comments.at[i, 'Comments']}")

            pnt.coords = [(df_cis_comments.at[i, 'Longitude'],
                           df_cis_comments.at[i, 'Latitude'], 0)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_COMMENTS
            pnt.style.iconstyle.scale = config.ICON_SCALE * 1.5
            pnt.style.labelstyle.scale = 0.5

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## -1.2 V

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 4  # Loop counter for type folders (4 = '-1.2 V')
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and config.PLOT_3D:
    df_cis_1200 = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    last_index = df_cis_1200.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_1200.at[i, 'Station'] < feet_counter:
            pnt = type_folders[k].newpoint(name='-1.200', visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = 'Potential: -1.2 V'
            pnt.coords = [(df_cis_1200.at[i, 'Longitude'], df_cis_1200.at[i, 'Latitude'],
                           1.2 * config.SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_1200
            pnt.style.iconstyle.scale = config.ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## -0.85 V

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 5  # Loop counter for type folders (5 = '-0.85 V')
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and config.PLOT_3D:
    df_cis_850 = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    last_index = df_cis_850.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_850.at[i, 'Station'] < feet_counter:
            pnt = type_folders[k].newpoint(name='-0.850', visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = 'Potential: -0.85 V'
            pnt.coords = [(df_cis_850.at[i, 'Longitude'], df_cis_850.at[i, 'Latitude'],
                           0.85 * config.SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_850
            pnt.style.iconstyle.scale = config.ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            pnt.style.linestyle.width = 0.01
            pnt.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, round(255 * 0.15))

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## ACVG

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 6  # Loop counter for type folders (6 = "ACVG Indication (dBV)")
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and 'ACVG Indication (dBV)' in df_cis.columns and config.PLOT_3D:
    df_cis_acvg = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    df_cis_acvg = df_cis_acvg[df_cis_acvg['ACVG Indication (dBV)'] != 0]
    df_cis_acvg = df_cis_acvg[['Station', 'Stationing (ft)', 'Latitude', 'Longitude',
                               'On Potential', 'Off Potential',
                               'ACVG Indication (dBV)', 'ACVG Notes']].reset_index(drop=True)
    last_index = df_cis_acvg.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_acvg.at[i, 'Station'] < feet_counter:
            # Break loop if there are no ACVG values
            if df_cis_acvg.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_acvg.at[i, 'ACVG Indication (dBV)'],
                                           visibility=config.DATA_VISIBILITY)

            if pd.isnull(df_cis_acvg.at[i, 'ACVG Notes']):
                pnt.style.balloonstyle.text = (
                    f"ACVG Indication: {df_cis_acvg.at[i, 'ACVG Indication (dBV)']} dB/V\n"
                    f"ID: {df_cis_acvg.index[i] + 1}\n"
                    f"Longitude: {df_cis_acvg.at[i, 'Longitude']}\n"
                    f"Latitude: {df_cis_acvg.at[i, 'Latitude']}\n"
                    f"Station (ft): {df_cis_acvg.at[i, 'Stationing (ft)']}")

            else:
                pnt.style.balloonstyle.text = (
                    f"ACVG Indication: {df_cis_acvg.at[i, 'ACVG Indication (dBV)']} dB/V\n"
                    f"ID: {df_cis_acvg.index[i] + 1}\n"
                    f"Longitude: {df_cis_acvg.at[i, 'Longitude']}\n"
                    f"Latitude: {df_cis_acvg.at[i, 'Latitude']}\n"
                    f"Station (ft): {df_cis_acvg.at[i, 'Stationing (ft)']}\n"
                    f"Comment: {df_cis_acvg.at[i, 'ACVG Notes']}")

            pnt.coords = [(df_cis_acvg.at[i, 'Longitude'], df_cis_acvg.at[i, 'Latitude'],
                           df_cis_acvg.at[i, 'ACVG Indication (dBV)'])]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_ACVG
            pnt.style.iconstyle.scale = config.ICON_SCALE + 0.5
            pnt.style.labelstyle.scale = 0

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## PCM (Amps)

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 7  # Loop counter for type folders (7 = 'PCM Data (Amps)')
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and 'PCM Data (Amps)' in df_cis.columns and config.PLOT_3D:
    df_cis_pcm_amps = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    df_cis_pcm_amps = df_cis_pcm_amps[df_cis_pcm_amps['PCM Data (Amps)'] != 0]
    df_cis_pcm_amps = df_cis_pcm_amps[['Station', 'Stationing (ft)', 'Latitude', 'Longitude',
                                       'PCM Data (Amps)']].reset_index(drop=True)
    last_index = df_cis_pcm_amps.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_pcm_amps.at[i, 'Station'] < feet_counter:
            # Break loop if there are no PCM (Amps) values
            if df_cis_pcm_amps.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_pcm_amps.at[i, 'PCM Data (Amps)'],
                                           visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"PCM: {df_cis_pcm_amps.at[i, 'PCM Data (Amps)']} Amps\n"
                f"Longitude: {df_cis_pcm_amps.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_pcm_amps.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_pcm_amps.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_pcm_amps.at[i, 'Longitude'], df_cis_pcm_amps.at[i, 'Latitude'],
                           df_cis_pcm_amps.at[i, 'PCM Data (Amps)'] * config.SCALE_PCM)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_PCM
            pnt.style.iconstyle.scale = config.ICON_SCALE + 0.5
            pnt.style.labelstyle.scale = 0

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## PCM (%)

In [ ]:
j = 0  # Loop counter for '.csv' files
k = 8  # Loop counter for type folders (8 = 'PCM % Change')
kmz_file = 0

# Goes through all '.csv' files
while j < len(original_data_list) and 'PCM % Change' in df_cis.columns and config.PLOT_3D:
    df_cis_pcm_percent = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))
    df_cis_pcm_percent = df_cis_pcm_percent[df_cis_pcm_percent['PCM % Change'] != 0]
    df_cis_pcm_percent = df_cis_pcm_percent[['Station', 'Stationing (ft)', 'Latitude', 'Longitude',
                                             'PCM % Change']].reset_index(drop=True)
    last_index = df_cis_pcm_percent.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_pcm_percent.at[i, 'Station'] < feet_counter:
            # Break loop if there are no PCM (%) values
            if df_cis_pcm_percent.shape[0] == 0:
                break

            pnt = type_folders[k].newpoint(name=df_cis_pcm_percent.at[i, 'PCM % Change'],
                                           visibility=config.DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"PCM: {df_cis_pcm_percent.at[i, 'PCM % Change']} %\n"
                f"Longitude: {df_cis_pcm_percent.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_pcm_percent.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_pcm_percent.at[i, 'Stationing (ft)']}")
            pnt.coords = [(df_cis_pcm_percent.at[i, 'Longitude'],
                           df_cis_pcm_percent.at[i, 'Latitude'],
                           config.SCALE_PCM_PERCENT +
                           df_cis_pcm_percent.at[i, 'PCM % Change'] * 0.5)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = config.ICON_PCM_PERCENT
            pnt.style.iconstyle.scale = config.ICON_SCALE + 0.5
            pnt.style.labelstyle.scale = 0

            # Counters
            i += 1

        # Export to '.kmz'
        cis_kmz[kmz_file].savekmz(kmz_path_list[kmz_file])

        # Counters
        current_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        kmz_file += 1
        k += total_folders

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

## Rename KMZs

In [ ]:
# TODO: Add seperation ---

In [ ]:
j = 0  # Loop counter for '.csv' files
kmz_file = 0
all_name_list = []
all_kmz_list = []

while j < len(total_miles_list):
    # List of SNXXXX+XX ranges
    sn_list = [''] * (math.ceil(total_miles_list[j]) * 2)

    # Last index
    last_index_sn = len(sn_list) - 1

    # List of '.kmz' file names
    name_list = [''] * (math.ceil(total_miles_list[j]))

    # List of stations as a nummber
    station_list = np.arange(0, math.ceil(total_miles_list[j]) + 1)

    # Last index
    last_index_station = len(station_list) - 1

    # First SN (number)
    station_list[0] = starting_stationing_list[j]

    # Assign a value of mile (ft) per element
    for i in range(1, last_index_station + 1):
        station_list[i] = station_list[i - 1] + 5280

    # Last SN (number)
    station_list[last_index_station] = (
        math.ceil(((total_miles_list[j] - math.floor(total_miles_list[j])) * 5280
                   + station_list[last_index_station - 1]))
    )

    #
    df_cis = pd.read_csv(os.path.join(DATA_DIR, f"{export_name_list[j]}(Modified).csv"))

    # First SN
    closest_index = (
        (df_cis['Station'] + starting_stationing_list[j] - station_list[0]).abs().argmin()
    )

    sn_list[0] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')

    s = 1  # station_list counter
    i = 1  # sn_list counter

    # In between SN
    while i < last_index_sn:
        closest_index = ((df_cis['Station'] + starting_stationing_list[j] - station_list[s]).abs()
                         .argmin())

        if df_cis['Station'].loc[closest_index] - station_list[s] >= 0:
            # Below closest index
            sn_list[i] = f"SN{df_cis['Stationing (ft)'].loc[closest_index - 1]}".replace(' ', '')
            sn_list[i + 1] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')

        else:
            # Above closest index
            sn_list[i] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')
            sn_list[i + 1] = f"SN{df_cis['Stationing (ft)'].loc[closest_index + 1]}".replace(' ',
                                                                                             '')

        # Counters
        s += 1
        i += 2

    # Last SN
    closest_index = (df_cis['Station'] + starting_stationing_list[j] -
                     station_list[last_index_station]).abs().argmin()
    sn_list[last_index_sn] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')

    # XXXXXXXXXXXXXXXXXXXXXXXXXXXXX

    # Remove duplicate SNs
    sn_list = list(OrderedDict.fromkeys(sn_list))

    # Get new index
    last_index_sn = len(sn_list) - 1

    # Sort kmz files
    kmz_files = glob.glob(os.path.join(KMZ_DIR, export_name_list[j], '*.kmz'))
    kmz_files = [os.path.basename(file) for file in kmz_files]
    kmz_files = natsorted(kmz_files)

    s = 0  # sn_list counter

    # Name station number ranges
    for i in range(0, last_index_station):
        name_list[i] = (f"{original_data_list[j].split('SN')[0][:-1]}"
                        f"({sn_list[s]} TO {sn_list[s + 1]}).kmz")

        # Counter
        s += 2

    all_name_list.extend(name_list)
    all_kmz_list.extend(kmz_files)

    # Counter
    j += 1

for i in range(0, len(kmz_path_list)):
    os.rename(kmz_path_list[i], kmz_path_list[i].replace(all_kmz_list[i], all_name_list[i]))

# Exception Report

In [ ]:
# TODO: Add to exception report when Off is more negative than On
# TODO: Add to exception report where skips exist
# TODO: Add to exception report between TS 

## Less Negative than -0.85V "On"

In [ ]:
j = 0  # Loop counter for '.csv' files

while j < len(total_miles_list):
    df_1 = exception_report_df(DATA_DIR, export_name_list[j]).copy()

    if not df_1.empty:
        df_1 = exception_report_crossings(df_1, threshold=Config.POTENTIAL_850)
        df_2 = exception_report_to_excel(df_1, df_cis=df_cis, acvg_threshold=45.0)
        # 
        file_path = Path(DATA_DIR) / f"{export_name_list[j]}(Exception Report).xlsx"
        with pd.ExcelWriter(file_path, mode="w", engine="openpyxl") as writer:
            df_2.to_excel(
                writer,
                startrow=4,
                sheet_name="Less Negative than -0.85V (On)",
                index=False,
            )
        last_index = len(df_2) * 2 - 1

        exception_report_format_excel(
            file_path=file_path,
            export_name=export_name_list[j],
            original_name=original_data_list[j],
            client=config.CLIENT,
            total_miles_list=total_miles_list,
            last_index=last_index,
            df_lengths=df_2,
        )

        # Hand off
        dfs[('-0.85V (On)', export_name_list[j])] = df_1
        dfs[('-0.85V (On)(Report)', export_name_list[j])] = df_2

    # Counter
    j += 1

## Less Negative than -0.85V "Off"

In [ ]:
# Combines all '.csv' files at specified folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(DATA_DIR, r'*.csv'))), ignore_index=True)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = (df_cis_filtered[df_cis_filtered.columns.intersection(
    ['Station', 'Stationing (ft)', 'Longitude', 'Latitude', 'Off Potential',
     'ACVG Indication (dBV)'])].reset_index(drop=True))
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [ ]:
i = 1  # Loop counter for rows

if df_cis_filtered.shape[0] != 0:
    # Finds crossing points
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'Off Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'Off Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'Off Potential'] >= -0.85):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # _'_
        elif ((df_cis_filtered.at[i - 1, 'Off Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -0.85) and
              (df_cis_filtered.at[i + 1, 'Off Potential'] > -0.85) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -0.85)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] < -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -0.85)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude']
    )
    df_cis_report['Length (ft)'] = 0.0

    if 'ACVG Indication (dBV)' in df_cis.columns:
        df_cis_report['ACVG Max (dB/V)'] = 0.0

    else:
        df_cis_report['ACVG Max (dB/V)'] = ''

    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # ACVG
        if 'ACVG Indication (dBV)' in df_cis.columns:
            mask = ((df_cis['Station'] >= df_cis_filtered.at[i, 'Station']) &
                    (df_cis['Station'] <= df_cis_filtered.at[i + 1, 'Station']) &
                    (df_cis['ACVG Indication (dBV)'] >= 45))

            df_cis_report.iat[j, 9] = df_cis.loc[mask, 'ACVG Indication (dBV)'].max()

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Export to excel
    writing_mode = 'w'

    if os.path.exists(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx")):
        writing_mode = 'a'

    with pd.ExcelWriter(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx"),
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4, sheet_name='Less Negative than -0.85V (Off)',
                               index=False)

In [ ]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx"))
    sheet = wb['Less Negative than -0.85V (Off)']

    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{config.CLIENT} ({original_data_list[0].split(' (SN')[0]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{original_data_list[0].split('SN')[1].split()[0]} to "
        f"{original_data_list[0].split('SN')[2].split('.')[0]}"
    )

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )
    total_length = round(df_cis_report['Length (ft)'].sum(), 2)
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance less negative than -0.85V 'Off' = "
        f"{total_length} feet ({length_percent} %)"
    )

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(4, 9).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:I1')
    sheet.merge_cells('A2:I2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')
    sheet.cell(4, 9).value = sheet.cell(5, 9).value
    sheet.merge_cells('I4:I5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:I' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:I' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:I' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFGH':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['I'].width = 50

In [ ]:
# Export to excel
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx"))

## More Negative than -1.2V "Off"

In [ ]:
# Combines all '.csv' files at specified folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(DATA_DIR, r'*.csv'))),
    ignore_index=True
)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                                   'Off Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [ ]:
i = 1  # Loop counter for rows

# Finds crossing points
if df_cis_filtered.shape[0] != 0:
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'Off Potential'] <= -1.2:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'Off Potential'] <= -1.2:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'Off Potential'] <= -1.2):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # '_'
        elif ((df_cis_filtered.at[i - 1, 'Off Potential'] < -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -1.2) and
              (df_cis_filtered.at[i + 1, 'Off Potential'] < -1.2) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] > -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -1.2)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] < -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -1.2)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude']
    )
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Export to excel
    writing_mode = 'w'

    if os.path.exists(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx")):
        writing_mode = 'a'

    with pd.ExcelWriter(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx"),
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4, sheet_name='More Negative than -1.2V (Off)',
                               index=False)

In [ ]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx"))
    sheet = wb['More Negative than -1.2V (Off)']

    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{config.CLIENT} ({original_data_list[0].split(' (SN')[0]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{original_data_list[0].split('SN')[1].split()[0]} to "
        f"{original_data_list[0].split('SN')[2].split('.')[0]}"
    )

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )
    total_length = round(df_cis_report['Length (ft)'].sum(), 2)
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance more negative than -1.2V 'Off' = "
        f"{total_length} feet ({length_percent} %)"
    )

In [ ]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

In [ ]:
# Save to excel
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(DATA_DIR, f"{export_name_list[0]}(Exception Report).xlsx"))

# ACVG Report

In [ ]:
if not config.PLOT_3D:
    # Dataframe
    df_cis_acvg = pd.read_csv(os.path.join(DATA_DIR, original_data_list[0]))
    df_cis_acvg = df_cis_acvg[df_cis_acvg['ACVG Indication (dBV)'] != 0]
    df_cis_acvg = df_cis_acvg[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                               'On Potential', 'Off Potential',
                               'ACVG Indication (dBV)', 'ACVG Notes']].reset_index(drop=True)

In [ ]:
# Borders
thin = Side(border_style='thin', color='000000')
white_border = Side(border_style='thin', color='FFFFFF')

## -0.85V "On" (ACVG >= 45)

In [ ]:
if 'ACVG Indication (dBV)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_on_45 = df_cis_acvg.copy()
    df_cis_acvg_on_45 = df_cis_acvg_on_45[(df_cis_acvg_on_45['On Potential'] >= -0.85) &
                                          (df_cis_acvg_on_45['ACVG Indication (dBV)'] >= 45.0)]
    df_cis_acvg_on_45 = df_cis_acvg_on_45.drop(columns='Station')
    last_index = df_cis_acvg_on_45.reset_index().last_valid_index()

    # Export to excel
    acvg_excel = f"ACVG Report ({export_name_list[0]}).xlsx"
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='w',
                        engine='openpyxl') as writer:
        df_cis_acvg_on_45.to_excel(writer, sheet_name='-0.85V (On)(ACVG 45)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (On)(ACVG 45)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

## -0.85V "On" (ACVG >= 60)

In [ ]:
if 'ACVG Indication (dBV)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_on_60 = df_cis_acvg.copy()
    df_cis_acvg_on_60 = df_cis_acvg_on_60[(df_cis_acvg_on_60['On Potential'] >= -0.85) &
                                          (df_cis_acvg_on_60['ACVG Indication (dBV)'] >= 60.0)]
    df_cis_acvg_on_60 = df_cis_acvg_on_60.drop(columns='Station')
    last_index = df_cis_acvg_on_60.reset_index().last_valid_index()

    # Save to excel
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='a',
                        engine='openpyxl') as writer:
        df_cis_acvg_on_60.to_excel(writer, sheet_name='-0.85V (On)(ACVG 60)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (On)(ACVG 60)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

## -0.85V "Off" (ACVG >= 45)

In [ ]:
if 'ACVG Indication (dBV)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_off_45 = df_cis_acvg.copy()
    df_cis_acvg_off_45 = df_cis_acvg_off_45[(df_cis_acvg_off_45['Off Potential'] >= -0.85) &
                                            (df_cis_acvg_off_45['ACVG Indication (dBV)'] >= 45.0)]
    df_cis_acvg_off_45 = df_cis_acvg_off_45.drop(columns='Station')
    last_index = df_cis_acvg_off_45.reset_index().last_valid_index()

    # Save to excel
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='a',
                        engine='openpyxl') as writer:
        df_cis_acvg_off_45.to_excel(writer, sheet_name='-0.85V (Off)(ACVG 45)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (Off)(ACVG 45)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

## -0.85V "Off" (ACVG >= 60)

In [ ]:
if 'ACVG Indication (dBV)' in df_cis.columns:
    # Dataframe
    df_cis_acvg_off_60 = df_cis_acvg.copy()
    df_cis_acvg_off_60 = df_cis_acvg_off_60[(df_cis_acvg_off_60['Off Potential'] >= -0.85) &
                                            (df_cis_acvg_off_60['ACVG Indication (dBV)'] >= 60.0)]
    df_cis_acvg_off_60 = df_cis_acvg_off_60.drop(columns='Station')
    last_index = df_cis_acvg_off_60.reset_index().last_valid_index()

    # Save to excel
    with pd.ExcelWriter(os.path.join(OUTPUT_DIR, acvg_excel), mode='a',
                        engine='openpyxl') as writer:
        df_cis_acvg_off_60.to_excel(writer, sheet_name='-0.85V (Off)(ACVG 60)', index_label='ID')

    # Modify sheet
    wb = load_workbook(os.path.join(OUTPUT_DIR, acvg_excel))
    sheet = wb['-0.85V (Off)(ACVG 60)']

    # Rename ACVG column
    sheet.cell(1, 7).value = 'ACVG (dB/V)'

    # Formatting
    i = 0

    while i < last_index + 2:
        for c in sheet['A1:H' + str(last_index + 2)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
            c.font = Font(bold=False)
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Header
    for i in range(1, 9):
        sheet.cell(1, i).font = Font(bold=True)

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

    # Save to excel
    wb.save(os.path.join(OUTPUT_DIR, acvg_excel))

# Severity Matrix

In [ ]:
# TODO: Deal with this when there is no acvg, etc.

In [ ]:
# # Dataframe
# df_cis_matrix = df_cis.copy()
# df_cis_matrix = df_cis_matrix[['Station', 'Stationing (ft)', 'On Potential', 'Off Potential',
#                                'Comments', 'Longitude', 'Latitude',
#                                'ACVG Indication (dBV)', 'ACVG Notes',
#                                'PCM Data (Amps)', 'PCM % Change']].reset_index(drop=True)
# df_cis_matrix['ACVG Severity'] = 0
# df_cis_matrix['PCM Severity'] = 0
# df_cis_matrix['Total Severity'] = 0

In [ ]:
# # ACVG
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dBV)'] < 50), 'ACVG Severity'] = 0
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dBV)'] >= 50) & (
#         df_cis_matrix['ACVG Indication (dBV)'] < 66), 'ACVG Severity'] = 1
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dBV)'] >= 66) & (
#         df_cis_matrix['ACVG Indication (dBV)'] < 81), 'ACVG Severity'] = 4
# df_cis_matrix.loc[(df_cis_matrix['ACVG Indication (dBV)'] >= 81), 'ACVG Severity'] = 6
# # df_cis_matrix = df_cis_matrix[df_cis_matrix['ACVG Indication (dBV)'] != 0]

In [ ]:
# # PCM
# df_cis_matrix.loc[(df_cis_matrix['PCM % Change'] >= 0) & (
#         df_cis_matrix['PCM % Change'] < 10), 'PCM Severity'] = 1
# df_cis_matrix.loc[(df_cis_matrix['PCM % Change'] >= 10) & (
#         df_cis_matrix['PCM % Change'] < 20), 'PCM Severity'] = 2
# df_cis_matrix.loc[(df_cis_matrix['PCM % Change'] >= 20), 'PCM Severity'] = 3
# # df_cis_matrix = df_cis_matrix[df_cis_matrix['PCM % Change'] != 0]

# Log File

In [ ]:
j = 0  # Loop counter for '.csv' files

# Current date and time
e = datetime.datetime.now()
execution_time = time.time() - start_time
minutes, seconds = divmod(execution_time, 60)

while j < len(original_data_list):
    total_rows_dropped = log_dict[export_name_list[j]]['Dataframe']['Rows Dropped']
    mean = log_dict[export_name_list[j]]['Statistics']['Mean']
    mode = log_dict[export_name_list[j]]['Statistics']['Mode']
    std = log_dict[export_name_list[j]]['Statistics']['Std']
    total_count = log_dict[export_name_list[j]]['Statistics']['Total Count']
    cutoff_count = log_dict[export_name_list[j]]['Statistics']['Cutoff Count']
    outliers_count = log_dict[export_name_list[j]]['Statistics']['Outliers Count']
    duplicates_count = log_dict[export_name_list[j]]['Statistics']['Duplicates Count']
    on_measurements = log_dict[export_name_list[j]]['Measurements']['On']
    off_measurements = log_dict[export_name_list[j]]['Measurements']['Off']

    # Modify log file with information of interest
    with open(os.path.join(LOGS_DIR, f"{export_name_list[j]}(Log).txt"), 'w') as f:
        f.write('-' * 75 + '\n')
        f.write(f"GPS DATA ({config.LOWER_CUTOFF}ft to {config.UPPER_CUTOFF}ft)\n")
        f.write('-' * 75 + '\n\n')
        f.write(f"Rows Dropped (Missing GPS): ")
        f.write(f"{total_rows_dropped}\n")
        f.write(f"Count: {total_count}\n")
        f.write(f"Cutoff Count: {total_count - cutoff_count}\n")
        f.write(f"Consistency: {round(cutoff_count / total_count * 100, 2)}%\n")
        f.write(f"Mean: {round(mean, 3)}\n")
        f.write(f"Mode: {mode}\n")
        f.write(f"Standard Deviation: {round(std, 3)}\n")

        if mean - std * 2.576 <= 0:
            f.write(f"99% Confidence Interval: {0} to {round(mean + std * 2.576, 3)}\n")

        else:
            f.write(f"99% Confidence Interval: {round(mean - std * 2.576, 3)} to "
                    f"{round(mean + std * 2.576, 3)}\n")

        f.write(f"Outliers: {outliers_count}\n")

        if config.PLOT_3D:
            f.write(
                f"Duplicated GPS Pairs: {duplicates_count}\n\n")

        if config.PLOT_3D:
            f.write('-' * 75 + '\n')
            f.write(f"MEASUREMENTS\n")
            f.write('-' * 75 + '\n\n')
            f.write(f"On: {on_measurements}\n")
            f.write(f"Off: {off_measurements}\n")
            f.write(f"Ratio (On/Off): {round(on_measurements / off_measurements, 3)}\n\n")

        if config.PLOT_3D:
            f.write('-' * 75 + '\n')
            f.write(f"VARIABLES\n")
            f.write('-' * 75 + '\n\n')
            f.write(f"Client: {config.CLIENT}\n")
            f.write(f"Reverse: {config.REVERSE}\n")
            f.write(f"Scale Factor: {config.SCALE_FACTOR}\n")
            f.write(f"Scale PCM: {config.SCALE_PCM}\n")
            f.write(f"Scale PCM (%): {config.SCALE_PCM_PERCENT}\n")
            f.write(f"Icon Scale: {config.ICON_SCALE}\n")
            f.write(f"Color Scheme: {config.COLOR_SCHEME}\n\n")

        f.write('-' * 75 + '\n')
        f.write(f"EXECUTION\n")
        f.write('-' * 75 + '\n\n')
        f.write(f"User: {socket.gethostname()}\n")
        f.write(f'{e.strftime("%b, %d, %Y")}, {e.hour:02d}:{e.minute:02d}:{e.second:02d}')
        f.write('\n')
        f.write(f"Execution Time ({int(minutes)} min {int(seconds)} sec)")

    # Counter
    j += 1